In [ ]:
!pip install autogen "httpx==0.27.2"

In [ ]:
import autogen
from autogen import AssistantAgent, UserProxyAgent
import os

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # openai 키 입력
llm_config = {"model" : "gpt-4o", "api_key" : os.environ["OPENAI_API_KEY"]}

In [ ]:
# AssistantAgent를 생성
assistant = AssistantAgent("assistant", llm_config = llm_config)

# UserProxyAgent를 생성
proxy = UserProxyAgent(
	"proxy", code_execution_config = {"executor" : autogen.coding.LocalCommandLineCodeExecutor(work_dir = "coding")}
)

In [ ]:
proxy.initiate_chat(
	assistant,
	message="1부터 10000까지 소수의 개수를 세는 코드를 알려줘.",
)
# proxy 객체를 사용하여 assistant 에이전트와 대화를 시작

In [ ]:
!pip install autogen "httpx==0.27.2"

In [ ]:
import autogen
from autogen import AssistantAgent, UserProxyAgent

import os
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # openai 키 입력

config_list = [{"model" : "gpt-4o", "api_key" : os.environ["OPENAI_API_KEY"]}]
llm_config = {
	"cache_seed" : 42, # 결과를 항상 일정하게 만들어주는 값
	"temperature" : 0, # 답변의 다양성을 설정하는 변수
	"config_list" : config_list, # API 키와 모델 정보를 전달
	"timeout" : 160, # 요청이 처리될 최대 시간(초)
}

In [ ]:
# 운동화 신제품 광고를 위해 공통, 제품 디자인, 마케팅 팀의 의견 제시
# 공통/일반 작업에 대한 프롬프트
task = '''
**Task**: 운동화 신제품 광고를 위해 다음 요구 사항에 맞는 솔루션을 제시해야 합니다.
- 연령, 성별, 소득 수준, 라이프스타일, 취미 등 세분화
- 고객의 운동화 사용 목적(스포츠, 패션, 일상 등) 분석.
- 운동화 제품군 내 다른 제품들과의 차별화 전략.
- 경쟁사 신제품 및 광고 전략 분석.
- 현재 시장 트렌드와 소비자 선호도 파악.
- 광고 제작, 촬영, 유통 채널별 비용 배분.
- 인적 자원 및 외부 에이전시 활용 계획.
- 신제품 출시 일정과 광고 캠페인 시점의 조율.
'''

# 제품 디자인팀에서 고려해야 할 내용에 대한 프롬프트
design_prompt = '''
**Role** : 당신은 제품 디자인팀의 팀원입니다.
다음의 요구 사항을 기반으로 제춤 디자인팀의 의견을 제시합니다:

- 제품의 주요 특징 및 USP(Unique Selling Point) 도출
- 제품의 세부 디테일과 장점을 강조할 수 있는 촬영 준비.
- 초기 광고 시안에서 제기된 소비자 테스트 피드백 반영.
- 운동화의 다양한 코디 및 용도(운동, 캐주얼, 정장 등)를 표 형태로 제시.
'''
design_prompt += task

# 마케팅 팀에서 고려해야 할 내용에 대한 프롬프트
marketing_prompt = '''
**Role**: 당신은 마케팅 팀의 팀원입니다.
온라인 & 오프라인 마케팅을 고려하지 않고 다양한 마케팅 행사를 제안해야 합니다.

- 광고 메시지 개발 : 슬로건 및 핵심 메시지 작성.
- 온라인 & 오프라인 채널별 전략을 비교하여 표 형태로 정리
- 광고 도달률, 클릭률, 전환율 등 성과 측정.
- 초기 프로모션(할인, 번들 판매 등) 준비.
- 광고 후 고객 피드백 및 리뷰 관리.
'''
marketing_prompt += task

# PM이 고려해야 할 내용에 대한 프롬프트
pm_prompt = '''
**Role**: 당신은 디자인팀과 마케팅 팀의 의견을 수렴하는 PM입니다.

- 당신은 제품 디자인팀과 마케팅팀이 제시한 의견의 단점을 지적하거나 추가 작업을 요청해야 합니다.
- 각 의견을 검토한 후, 비즈니스 요구 사항에 따라 최상의 광고 전략을 도출해야 합니다.
- 결정을 위해 필요한 만큼의 요약 표를 사용할 수 있습니다.
'''
pm_prompt += task

In [ ]:
user_proxy = UserProxyAgent(
	name="proxy",
	system_message = "Hub for User and AssistantAgent", # 사용자와 AssistantAgent 간의 허브 역할
  code_execution_config={
	  "last_n_messages" : 2, # 최근 2개의 메시지만 기억
	  "workd_dir" : "multiagent", # 그룹 채팅 작업 디렉터리를 설정
	 },
	 human_input_mode="NEVER", # 사용자 입력을 받지 않고 자동으로 작업 처리
) # 다른 에이전트들과 협력하여 메시지를 전달하거나 요청을 처리

design_agent = AssistantAgent(
	name = "design",
	system_message = design_prompt, # 디자인팀과 관련된 초기 프롬프트를 제공하여 에이전트 역할을 지정
	llm_config = {"config_list": config_list} # LLM 호출
	) # 디자인 관련 요청을 처리하는 에이전트(제품 디자인팀)

marketing_agent = AssistantAgent(
	name = "marketing",
	system_message = marketing_prompt, # 마케팅팀과 관련된 초기 프롬프트를 제공하여 에이전트 역할을 지정
	llm_config = {"config_list": config_list} # LLM 호출
	) # 마케팅 관련 요청을 처리하는 에이전트(마케팅팀)

pm_agent = AssistantAgent(
	name = "pm",
	system_message = pm_prompt, # pm 역할을 수행하도록 초기 프롬프트를 제공
	llm_config = {"config_list": config_list} # LLM 호출
	) # 홍보 PM으로서 다른 에이전트의 작업을 관리하고 결과를 조정(PM)

In [ ]:
# latest_speaker : 대화에서 마지막으로 발언한 에이전트(현재 상태), groupchat: 대화의 전체 컨텍스트를 담고 있는 객체
def state_transition(lastest_speaker, groupchat):
	messages = groupchat.messages
	if lastest_speaker is user_proxy:
		return design_agent # 마지막 화자가 user_proxy라면 다음 상태로 design_agent를 반환
	elif lastest_speaker is design_agent:
		return marketing_agent # 마지막 화자가 design_agent라면 다음 상태로 marketing_agent를 반환
	elif lastest_speaker is marketing_agent:
		return pm_agent # 마지막 화자가 marketing_agent라면 다음 상태로 pm_agent를 반환
	elif lastest_speaker is pm_agent:
		return None # 마지막 화자가 pm_agent라면 더 이상 상태 전환이 없으며, None을 반환하여 대화가 종료

In [ ]:
groupchat = autogen.GroupChat(
    agents = [user_proxy, design_agent, marketing_agent, pm_agent], # 대화에 참여하는 에이전트들의 리스트
    messages = [], # 초기 메시지의 리스트로, 비어 있는 상태([]) 로 시작
    max_round = 4, # 대화의 최대 라운드 수(무한 루프를 방지)
    speaker_selection_method = state_transition, # 대화에서 다음 화자를 결정하는 함수
    )

manager = autogen.GroupChatManager(
    groupchat = groupchat,
    llm_config = llm_config
) # GroupChatManager : 그룹 채팅의 전체 흐름을 관리하는 객체

user_proxy.initiate_chat(
    manager,
    message="이 비즈니스 요구 사항을 기반으로 최상의 운동화 광고 전략을 제안하세요."
)   # user_proxy가 초기 메시지를 보냄으로써 대화가 시작